In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE

In [3]:
df_sales = pd.read_csv('Sales_Data05.csv')
df_member = pd.read_csv('Member_Data.csv',encoding='cp949')
df_product = pd.read_csv('Product_Data.csv')

In [5]:
# 1. 데이터 전처리 및 고객별 피처 엔지니어링
# =============================================================================

# --- df_sales 전처리 ---
# 주문일시와 배송완료일을 datetime으로 변환
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'], errors='coerce')
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce')

# 배송기간 계산 (일수): 배송완료일 - 주문일시
df_sales['배송기간'] = (df_sales['배송완료일'] - df_sales['주문일시']).dt.days

# 배송지연여부 생성: 배송기간이 2일 초과하면 1, 아니면 0
delay_threshold = 2
df_sales['배송지연여부'] = (df_sales['배송기간'] > delay_threshold).astype(int)

# --- 고객별 집계 ---
# (a) 총 주문 건수와 총 구매 금액, 배송지연 주문 수 집계 (df_sales 기준)
customer_orders = df_sales.groupby('회원번호').agg(
    total_orders=('회원번호', 'count'),
    total_purchase_amount=('구매금액', 'sum'),
    shipping_delay_count=('배송지연여부', 'sum')
).reset_index()
customer_orders['shipping_delay_freq'] = customer_orders['shipping_delay_count'] / customer_orders['total_orders']

# (b) 고객별 평균 구매 주기 계산 (일 단위)
def compute_avg_cycle(dates):
    dates = dates.sort_values()
    if len(dates) < 2:
        return np.nan
    diffs = dates.diff().dropna().dt.days
    return diffs.mean()

purchase_cycle = df_sales.groupby('회원번호')['주문일시'].apply(compute_avg_cycle).reset_index(name='avg_purchase_cycle')

# (c) 고객별 마지막 주문일 계산
last_order = df_sales.groupby('회원번호')['주문일시'].max().reset_index(name='last_order_date')

# (d) 고객 피처 병합: 주문 집계, 평균 구매 주기, 마지막 주문일
customer_features = customer_orders.merge(purchase_cycle, on='회원번호', how='left')
customer_features = customer_features.merge(last_order, on='회원번호', how='left')

# --- df_member 전처리 ---
# 구독여부 전처리: 문자열 소문자화, 공백 제거 및 매핑 ('true', 'yes' -> True; 'false', 'no' -> False)
df_member['구독여부'] = df_member['구독여부'].astype(str).str.strip().str.lower()
mapping_dict = {'true': True, 'yes': True, 'false': False, 'no': False}
df_member['구독여부'] = df_member['구독여부'].map(mapping_dict)

# --- 고객 피처에 회원 정보(구독여부) 병합 ---
customer_features = customer_features.merge(df_member[['회원번호', '구독여부']], on='회원번호', how='left')

# 결측치 처리: avg_purchase_cycle의 NaN은 중앙값으로 대체
customer_features['avg_purchase_cycle'] = customer_features['avg_purchase_cycle'].fillna(customer_features['avg_purchase_cycle'].median())


In [6]:
# 2. 라벨 생성: 목표 변수 설정
# =============================================================================

# 모델 1: 구독 확률 예측
# 이미 customer_features에 '구독여부' 컬럼이 있으므로, 목표 변수는 그대로 사용.
# NaN 처리: 구독여부가 NaN인 경우 미구독(0)으로 처리
customer_features['구독여부'] = customer_features['구독여부'].fillna(False).astype(int)

# 모델 2: 자주 구매(구매주기 n개월 이하) 예측
# 기준: n개월을 일(day)로 환산 (예: n=3개월 -> 약 90일 이하)
n_months = 3
threshold_days = n_months * 30  # 약 90일
customer_features['frequent_buyer'] = (customer_features['avg_purchase_cycle'] <= threshold_days).astype(int)


C:\Users\sylvi\AppData\Local\Temp\ipykernel_11588\3427967024.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  customer_features['구독여부'] = customer_features['구독여부'].fillna(False).astype(int)


In [7]:
# 3. 모델 입력 변수 구성
# =============================================================================

# 선택 피처: 구매 내역, 구매 주기, 배송지연 빈도
feature_cols = ['total_orders', 'total_purchase_amount', 'avg_purchase_cycle', 'shipping_delay_freq']

# -----------------------
# 모델 1: 구독 확률 예측 모델 (목표: '구독여부')
# -----------------------
X_sub = customer_features[feature_cols]
y_sub = customer_features['구독여부']

X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_sub, y_sub, test_size=0.3, random_state=42)

# 오버샘플링: SMOTE 적용
smote = SMOTE(random_state=42)
X_train_sub_res, y_train_sub_res = smote.fit_resample(X_train_sub, y_train_sub)

print("구독 모델 - 원본 학습 데이터 클래스 분포:")
print(y_train_sub.value_counts())
print("구독 모델 - 오버샘플링 후 학습 데이터 클래스 분포:")
print(pd.Series(y_train_sub_res).value_counts())

# 로지스틱 회귀 모델 학습 (class_weight='balanced' 추가)
log_reg_sub = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg_sub.fit(X_train_sub_res, y_train_sub_res)

y_pred_sub = log_reg_sub.predict(X_test_sub)
y_pred_proba_sub = log_reg_sub.predict_proba(X_test_sub)[:, 1]

print("\n----- 구독 확률 예측 모델 (오버샘플링 적용) -----")
print("혼동 행렬:")
print(confusion_matrix(y_test_sub, y_pred_sub))
print("\n분류 리포트:")
print(classification_report(y_test_sub, y_pred_sub))
print("ROC AUC Score (구독 모델):", roc_auc_score(y_test_sub, y_pred_proba_sub))

# -----------------------
# 모델 2: 자주 구매 (구매주기 n개월 이하) 예측 모델 (목표: 'frequent_buyer')
# -----------------------
X_freq = customer_features[feature_cols]
y_freq = customer_features['frequent_buyer']

X_train_freq, X_test_freq, y_train_freq, y_test_freq = train_test_split(X_freq, y_freq, test_size=0.3, random_state=42)

# 자주 구매 고객의 경우, 극단적인 불균형이 있을 수 있으므로 SMOTE 적용
smote_freq = SMOTE(random_state=42)
X_train_freq_res, y_train_freq_res = smote_freq.fit_resample(X_train_freq, y_train_freq)

print("\n자주 구매 모델 - 원본 학습 데이터 클래스 분포:")
print(y_train_freq.value_counts())
print("자주 구매 모델 - 오버샘플링 후 학습 데이터 클래스 분포:")
print(pd.Series(y_train_freq_res).value_counts())

log_reg_freq = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg_freq.fit(X_train_freq_res, y_train_freq_res)

y_pred_freq = log_reg_freq.predict(X_test_freq)
y_pred_proba_freq = log_reg_freq.predict_proba(X_test_freq)[:, 1]

print("\n----- 자주 구매 (구매주기 n개월 이하) 예측 모델 (오버샘플링 적용) -----")
print("혼동 행렬:")
print(confusion_matrix(y_test_freq, y_pred_freq))
print("\n분류 리포트:")
print(classification_report(y_test_freq, y_pred_freq))
print("ROC AUC Score (자주 구매 모델):", roc_auc_score(y_test_freq, y_pred_proba_freq))


구독 모델 - 원본 학습 데이터 클래스 분포:
구독여부
0    7549
1    1229
Name: count, dtype: int64
구독 모델 - 오버샘플링 후 학습 데이터 클래스 분포:
구독여부
0    7549
1    7549
Name: count, dtype: int64

----- 구독 확률 예측 모델 (오버샘플링 적용) -----
혼동 행렬:
[[ 753 2485]
 [ 102  422]]

분류 리포트:
              precision    recall  f1-score   support

           0       0.88      0.23      0.37      3238
           1       0.15      0.81      0.25       524

    accuracy                           0.31      3762
   macro avg       0.51      0.52      0.31      3762
weighted avg       0.78      0.31      0.35      3762

ROC AUC Score (구독 모델): 0.511859408078684

자주 구매 모델 - 원본 학습 데이터 클래스 분포:
frequent_buyer
1    8689
0      89
Name: count, dtype: int64
자주 구매 모델 - 오버샘플링 후 학습 데이터 클래스 분포:
frequent_buyer
1    8689
0    8689
Name: count, dtype: int64

----- 자주 구매 (구매주기 n개월 이하) 예측 모델 (오버샘플링 적용) -----
혼동 행렬:
[[  46    0]
 [   0 3716]]

분류 리포트:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
      